### Import Nyoka

In [1]:
from nyoka import model_to_pmml

### Scripts

In [2]:
def script1():
    r3 = r1+r2
    
def script2():
    r6 = r1+r2+r3-r4

### Create a pipeline for preprocessing

In [3]:
import pandas as pd
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

iris = datasets.load_iris()
irisd = pd.DataFrame(iris.data, columns=iris.feature_names)
irisd['Species'] = iris.target

features = irisd.columns.drop('Species')
target = 'Species'

pipelineOnly = Pipeline([
    ('Scaling', StandardScaler()), 
    ('Imputing', Imputer())
])

pipelineOnly.fit(irisd[features])
Xdata = pipelineOnly.transform(irisd[features])

### sklearn model

In [5]:
rfObj = RandomForestClassifier()
rfObj.fit(Xdata,irisd['Species'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Exporting pipeline and model object seperately - sklearn

In [5]:
toExportDict={
    'model1':{
        'preProcessingScript':{'scripts':[script1,script2], 'scriptpurpose':['train','score']},
        'pipelineObj':pipelineOnly,
        'modelObj':rfObj,
        'featuresUsed':features,
        'targetName':'Species',
        'postProcessingScript':{'scripts':[script1], 'scriptpurpose':['postprocess']},
        'taskType': 'trainAndscore'
    }
}

pmml = model_to_pmml(toExportDict, pmml_f_name="sklearnppOnly.pmml")

### Keras model

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128, verbose=0)

C:\Users\vran\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Exporting pipeline and model object seperately - Keras

In [6]:
toExportDict={
    'model1':{
        'hyperparameters':None,
        'preProcessingScript':{'scripts':[script1,script2], 'scriptpurpose':['train','score']},
        'pipelineObj':pipelineOnly,
        'modelObj':model,
        'featuresUsed':features,
        'targetName':None,
        'postProcessingScript':{'scripts':[script1], 'scriptpurpose':['postprocess']},
        'taskType': 'trainAndscore'
    }
}

pmml = model_to_pmml(toExportDict, pmml_f_name="KerasppOnly.pmml")

### Exporting pipeline and model object seperately - Multiple models

In [8]:
toExportDict={
    'model1':{
        'preProcessingScript':{'scripts':[script1,script2], 'scriptpurpose':['train','score']},
        'pipelineObj':pipelineOnly,
        'modelObj':rfObj,
        'featuresUsed':features,
        'targetName':'Species',
        'postProcessingScript':{'scripts':[script1], 'scriptpurpose':['postprocess']},
        'taskType': 'trainAndscore'
    },
    'model2':{
        'preProcessingScript':{'scripts':[script1,script2], 'scriptpurpose':['train','score']},
        'pipelineObj':pipelineOnly,
        'modelObj':model,
        'featuresUsed':features,
        'targetName':None,
        'postProcessingScript':{'scripts':[script1], 'scriptpurpose':['postprocess']},
        'taskType': 'trainAndscore'
    },
}

pmml = model_to_pmml(toExportDict, pmml_f_name="MultipleModels.pmml")